In [22]:
import numpy as np
import pandas as pd

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.losses import CategoricalCrossentropy, KLDivergence
from keras.optimizers import Adam

from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

from pnn import get_initial_weights_with_ga, set_model_weights

In [23]:
DISTRIBUTION_FILE = "data/incidents_distribution_station.csv"
DISTRIBUTION_TEST_FILE = "data/incidents_distribution_station_avg_test.csv"

In [24]:
def get_model(nodes_in_hidden_layers, dropout, nr_of_inputs):
    model = Sequential()
    for i in range(len(nodes_in_hidden_layers)):
        if i == 0:
            model.add(Dense(nodes_in_hidden_layers[i], activation='relu', input_shape=(nr_of_inputs,)))  # Input and hidden layer
        else:
            if dropout != 0:
                model.add(Dropout(dropout))
            model.add(Dense(nodes_in_hidden_layers[i], activation='relu'))  # Additional hidden layers
    model.add(Dense(1, activation="linear"))  # Output layer
    model.compile(optimizer=Adam(learning_rate=0.0001), loss="mean_squared_error", metrics=("mean_squared_error"))
    return model

In [25]:
def cross_validation(x_train, y_train, nodes, epochs, dropout, patience):
    kf = KFold(n_splits=5)
    ce = []
    for train_index, validation_index in kf.split(x_train):
        model = get_model(nodes, dropout, len(x_train[0]))
        es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=patience)
        history = model.fit(x_train[train_index], y_train[train_index], validation_data=(x_train[validation_index], y_train[validation_index]), epochs=epochs, verbose=1, callbacks=[es])
        ce.append(min(history.history['mean_squared_error']))
    avg_ce = sum(ce) / len(ce)
    return avg_ce

In [27]:
def preprocess(data, scaler):
    data = data[data["Base Station"] == 8]
    data = data[data["Month"] == 8]

    #station = np.eye(19)[data["Base Station"]]
    #year = np.eye(4)[data["Year"] - 2015]
    month = np.eye(12)[data["Month"] - 1]
    #day = np.eye(31)[data["Day"] - 1]
    #week = np.eye(53)[data["Week"] - 1]
    weekday = np.eye(7)[data["Weekday"] - 1]
    hour = np.eye(24)[data["Hour"]]

    x = np.concatenate([
        #station,
        #year, 
        month, 
        #day, 
        #week, 
        weekday, 
        hour], axis=1)

    y = data.iloc[:, -1].to_numpy()
    y = np.array(y).reshape(-1, 1)
    y_scaled = scaler.fit_transform(y)
    y_scaled = y_scaled.flatten()
    y = y_scaled

    return x, y

In [28]:
data = pd.read_csv(DISTRIBUTION_FILE, encoding='utf-8', escapechar='\\', parse_dates=True)
data_test = pd.read_csv(DISTRIBUTION_TEST_FILE, encoding='utf-8', escapechar='\\', parse_dates=True)

In [29]:
scaler = MinMaxScaler()
X_train, Y_train = preprocess(data, MinMaxScaler())
X_test, Y_test = preprocess(data_test, scaler)

In [17]:
scaler = MinMaxScaler()
x, y = preprocess(data, scaler)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [30]:
layers = [64, 128, 32, 8]
model = get_model(layers, 0.5, len(X_train[0]))
#population = get_initial_weights_with_ga(base_model=model, train_x=X_train, train_y=Y_train, population_size=30, generations=20, verbose=True)
#model = set_model_weights(population, model, 0)

es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=15)
model.fit(X_train, Y_train, epochs=200, verbose=1, callbacks=[es])

Epoch 1/200
93/93 [==============================] - 0s 911us/step - loss: 0.1416 - mean_squared_error: 0.1416
Epoch 2/200
93/93 [==============================] - 0s 856us/step - loss: 0.0779 - mean_squared_error: 0.0779
Epoch 3/200
93/93 [==============================] - 0s 867us/step - loss: 0.0607 - mean_squared_error: 0.0607
Epoch 4/200
93/93 [==============================] - 0s 1ms/step - loss: 0.0515 - mean_squared_error: 0.0515
Epoch 5/200
93/93 [==============================] - 0s 867us/step - loss: 0.0435 - mean_squared_error: 0.0435
Epoch 6/200
93/93 [==============================] - 0s 856us/step - loss: 0.0385 - mean_squared_error: 0.0385
Epoch 7/200
93/93 [==============================] - 0s 900us/step - loss: 0.0353 - mean_squared_error: 0.0353
Epoch 8/200
93/93 [==============================] - 0s 856us/step - loss: 0.0350 - mean_squared_error: 0.0350
Epoch 9/200
93/93 [==============================] - 0s 846us/step - loss: 0.0321 - mean_squared_error: 0.0321
Epo

In [31]:
predictions = model.predict(X_test)

6/6 [==============================] - 0s 799us/step


In [32]:
#year = np.argmax(X_test[:, :4], axis=1) + 2015
month = np.argmax(X_test[:, :12], axis=1) + 1
#day = np.argmax(X_test[:, 16:47], axis=1) + 1
#week = np.argmax(X_test[:, 47:100], axis=1) + 1
weekday = np.argmax(X_test[:, 12:19], axis=1) + 1
hour = np.argmax(X_test[:, 19:], axis=1)

In [33]:
mse = mean_squared_error(Y_test, predictions)
print("Mean squared error scaled: ", mse)

predictions_inversed = scaler.inverse_transform(predictions)
Y_test_inversed = scaler.inverse_transform(Y_test.reshape(-1, 1))
mse = mean_squared_error(Y_test_inversed, predictions_inversed)
print("Mean squared error: ", mse)

""" h = [(round(p[0], 3), round(t[0], 3)) for p, t in zip(predictions_inversed, Y_test_inversed)]
for i in h:
    print(i) """

print("pred target year month day week weekday hour")
for i in range(len(predictions_inversed)):
    p = round(predictions_inversed[i][0], 3)
    t = round(Y_test_inversed[i][0], 3)
    # print(p, t, year[i], month[i], day[i], week[i], weekday[i], hour[i], Y_test_inversed[i])
    print(p, t, month[i], weekday[i], hour[i], Y_test_inversed[i])


Mean squared error scaled:  0.03536378419920199
Mean squared error:  0.8840946104563853
pred target year month day week weekday hour
0.554 1.0 8 1 0 [1.]
0.514 0.0 8 1 1 [0.]
0.504 1.0 8 1 2 [1.]
0.51 0.0 8 1 3 [0.]
0.507 1.0 8 1 4 [1.]
0.503 0.0 8 1 5 [0.]
0.496 0.0 8 1 6 [0.]
0.504 3.0 8 1 7 [3.]
0.525 0.0 8 1 8 [0.]
0.869 2.0 8 1 9 [2.]
1.121 2.0 8 1 10 [2.]
0.978 2.0 8 1 11 [2.]
1.032 1.0 8 1 12 [1.]
1.004 1.0 8 1 13 [1.]
1.004 2.0 8 1 14 [2.]
1.009 2.0 8 1 15 [2.]
1.013 2.0 8 1 16 [2.]
1.031 5.0 8 1 17 [5.]
1.115 1.0 8 1 18 [1.]
1.004 2.0 8 1 19 [2.]
0.984 1.0 8 1 20 [1.]
1.079 0.0 8 1 21 [0.]
0.582 0.0 8 1 22 [0.]
0.602 2.0 8 1 23 [2.]
0.527 2.0 8 2 0 [2.]
0.553 0.0 8 2 1 [0.]
0.517 1.0 8 2 2 [1.]
0.507 0.0 8 2 3 [0.]
0.499 2.0 8 2 4 [2.]
0.505 0.0 8 2 5 [0.]
0.498 2.0 8 2 6 [2.]
0.514 0.0 8 2 7 [0.]
0.529 0.0 8 2 8 [0.]
0.955 1.0 8 2 9 [1.]
1.047 1.0 8 2 10 [1.]
0.894 0.0 8 2 11 [0.]
0.904 1.0 8 2 12 [1.]
0.982 1.0 8 2 13 [1.]
0.854 1.0 8 2 14 [1.]
0.926 2.0 8 2 15 [2.]
0.889 0.

In [ ]:
avg_ce = cross_validation(X_train, Y_train, layers, 10, 0.2, 5)